<a href="https://colab.research.google.com/github/Kushmeet-123/Hindi_Speech_Emotion_Recognition./blob/main/speech_emotion_hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install kaggle librosa tensorflow fastapi uvicorn numpy pandas

In [4]:
from google.colab import files
files.upload()  # This will open a file picker to upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kushmeetkaur","key":"453702d1f16263be56e197067f20935e"}'}

In [6]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d vishlb/speech-emotion-recognition-hindi
!unzip speech-emotion-recognition-hindi.zip -d hindi_emotion_data

Dataset URL: https://www.kaggle.com/datasets/vishlb/speech-emotion-recognition-hindi
License(s): ODbL-1.0
 94% 281M/300M [00:05<00:01, 16.0MB/s]
100% 300M/300M [00:05<00:00, 56.7MB/s]
Archive:  speech-emotion-recognition-hindi.zip
  inflating: hindi_emotion_data/my Dataset/1/session1/anger/1.1.anger-01.wav  
  inflating: hindi_emotion_data/my Dataset/1/session1/anger/1.1.anger-02.wav  
  inflating: hindi_emotion_data/my Dataset/1/session1/anger/1.1.anger-03.wav  
  inflating: hindi_emotion_data/my Dataset/1/session1/anger/1.1.anger-04.wav  
  inflating: hindi_emotion_data/my Dataset/1/session1/anger/1.1.anger-05.wav  
  inflating: hindi_emotion_data/my Dataset/1/session1/anger/1.1.anger-06.wav  
  inflating: hindi_emotion_data/my Dataset/1/session1/anger/1.1.anger-07.wav  
  inflating: hindi_emotion_data/my Dataset/1/session1/anger/1.1.anger-08.wav  
  inflating: hindi_emotion_data/my Dataset/1/session1/anger/1.1.anger-09.wav  
  inflating: hindi_emotion_data/my Dataset/1/session1/ange

In [8]:
import os
import pandas as pd

dataset_path = 'hindi_emotion_data/'
print(os.listdir(dataset_path))

['my Dataset']


In [10]:
import os

# Define the path to your dataset
dataset_path = 'hindi_emotion_data/my Dataset/8/session5'

# List of emotion categories (adjust based on your dataset)
emotion_categories = ['happy', 'sad', 'fear', 'neutral', 'sarcastic', 'surprise']

# Prepare a dictionary to store file paths by emotion category
dataset = {}

# Load all file paths for each emotion category
for emotion_category in emotion_categories:
    emotion_folder = os.path.join(dataset_path, emotion_category)
    if os.path.exists(emotion_folder):
        # List all files in the emotion folder
        dataset[emotion_category] = [os.path.join(emotion_folder, file) for file in os.listdir(emotion_folder) if file.endswith('.wav')]  # Assuming files are WAV format
    else:
        print(f"Folder for {emotion_category} does not exist!")

# Display the loaded dataset (optional)
for emotion, files in dataset.items():
    print(f"{emotion}: {len(files)} files")


happy: 10 files
sad: 10 files
fear: 10 files
neutral: 10 files
sarcastic: 10 files
surprise: 10 files


In [11]:
import librosa
import numpy as np

# Function to extract MFCC features from an audio file
def extract_mfcc(audio_path):
    try:
        # Load the audio file
        y, sr = librosa.load(audio_path, sr=22050)  # y is the waveform, sr is the sample rate
        # Extract MFCC features from the audio file
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc = np.mean(mfcc.T, axis=0)  # Take the mean of the MFCCs along time axis
        return mfcc
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

# Prepare the dataset by extracting MFCC features from all audio files
features = []
labels = []

# Loop over the loaded dataset and extract features
for emotion_category, files in dataset.items():
    for file in files:
        mfcc = extract_mfcc(file)

        if mfcc is not None:
            features.append(mfcc)
            labels.append(emotion_category)  # Label is the emotion category (e.g., 'happy')

# Convert the features and labels into numpy arrays for model training
X = np.array(features)
y = np.array(labels)

# Display the shape of the extracted features and labels
print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")

# Example: Saving the features and labels for later use
np.save('features.npy', X)
np.save('labels.npy', y)

Features shape: (60, 13)
Labels shape: (60,)


In [36]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming you've already extracted features (MFCCs) and your dataset is loaded into X (features) and y (labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle class imbalance using SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Train the RandomForest model with class weights
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train_res, y_train_res)

# Predict on the test set
y_pred = model.predict(X_test)

# Print classification report (accuracy, precision, recall, F1-score)
print(classification_report(y_test, y_pred, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


              precision    recall  f1-score   support

        fear       0.00      0.00      0.00         0
       happy       0.00      0.00      0.00         2
     neutral       0.00      0.00      0.00         2
         sad       0.00      0.00      0.00         2
   sarcastic       1.00      0.33      0.50         3
    surprise       0.33      0.33      0.33         3

    accuracy                           0.17        12
   macro avg       0.22      0.11      0.14        12
weighted avg       0.33      0.17      0.21        12



In [37]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [38]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [39]:
%%writefile app.py
import streamlit as st
import librosa
import numpy as np
import pickle

# Load the model
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

st.title("Hindi Speech Emotion Recognition")

uploaded_file = st.file_uploader("Upload an audio file", type=["wav", "mp3"])
if uploaded_file is not None:
    y, sr = librosa.load(uploaded_file, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs.T, axis=0).reshape(1, -1)
    prediction = model.predict(mfccs_mean)
    st.write(f"Predicted Emotion: {prediction[0]}")

Writing app.py


In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.227.144.149:8501

